# <center>Big Data &ndash; Exercises</center>
## <center>Autumn 2025 &ndash; Week 5 &ndash; ETH Zurich</center>
## <center>Wide Column Stores - HBase</center>

## Overview

In this exercise you will analyse some important architectural components of HBase. The following is a brief summary of the exercises and their corresponding topics:

[Exercise 1: Region Servers](#exercise_1)

[Exercise 2: HFile Indices](#exercise_2)

[Exercise 3: Bloom Filters](#exercise_3)

[Exercise 4: Log-Structured Merge-Trees](#exercise_4)

[Exercise 5: Write-ahead Logs](#exercise_5)

<a id='exercise_1'></a>
## Exercise 1 &mdash; Inside a RegionServer

In this exercise, you will take on the role of a RegionServer in HBase and execute several `get` queries.

#### Setting
Imagine that we have an HBase table called '`phrases`', which has a single column family `words` with 3 columns: A, B and C. Each of these columns stores a single word.

Recall from the lecture slides that keys in HBase have the following structure:

<img src="https://polybox.ethz.ch/index.php/s/XjO0LM1r6L10kT3/download" width="60%">
<a id="simple_keys"></a>

To avoid excessive clutter in this exercise, we simplify the structure as follows:
- We omit the `column family` from the key, since the table has a single column family.
- We omit the `key length` and `column family length` fields.
- We omit the `key type` field, i.e. we assume no records can be deleted. 
- Instead of a number of milliseconds, the `timestamp` field will contain integers from 1 to 10, where a larger integer signifies a latter version.

Thus, keys that will be used in this exercise consist of only three fileds: **row, column, timestamp**. This is pictured below:

<img src="https://polybox.ethz.ch/index.php/s/82LbiSQCyTnMr5i/download" width="30%">

### Executing queries
Imagine that you are the RegionServer responsible for storing the region with row keys `100` - `999`.

The diagram below represents your state at the time of execution:

<img src="https://polybox.ethz.ch/index.php/s/mbc7boFDDCo2V9D/download" >

State which Key-Value pairs will be returned by each of the following queries. Assume that the HBase instance is configured to return only the latest version of a cell.

1. `get 'phrases', '278'`

| Row | Column | Timestamp | Value | Where it came from (which HFile) |
|:---:|:------:|:---------:|:------|:--------------------------------:|
|     |        |           |       |                                  |
|     |        |           |       |                                  |
|     |        |           |       |                                  |

2. `get 'phrases', '636'`

| Row | Column | Timestamp | Value | Where it came from (which HFile) |
|:---:|:------:|:---------:|:------|:--------------------------------:|
|     |        |           |       |                                  |
|     |        |           |       |                                  |
|     |        |           |       |                                  |

3. `get 'phrases', '593'`

| Row | Column | Timestamp | Value | Where it came from (which HFile) |
|:---:|:------:|:---------:|:------|:--------------------------------:|
|     |        |           |       |                                  |
|     |        |           |       |                                  |
|     |        |           |       |                                  |

4. `get 'phrases', '640'`

| Row | Column | Timestamp | Value | Where it came from (which HFile) |
|:---:|:------:|:---------:|:------|:--------------------------------:|
|     |        |           |       |                                  |
|     |        |           |       |                                  |
|     |        |           |       |                                  |

3. `get 'phrases', '443'`

| Row | Column | Timestamp | Value | Where it came from (which HFile) |
|:---:|:------:|:---------:|:------|:--------------------------------:|
|     |        |           |       |                                  |
|     |        |           |       |                                  |
|     |        |           |       |                                  |

<a id='exercise_2'></a>
## Exercise 2 &mdash; Building an HFile index

When servicing a Read request, the RegionServer needs to check its MemStore and all HFiles for the existence of the requested key. In order to avoid scanning HFiles entirely, HBase uses sparse index structures to quickly skip to the position of the *HBase block* which *may* hold the requested key. By default, each *HBase block* is 64kB (but it is configurable) in size and always contains whole key-value pairs, so, if a block needs more than 64kB to avoid splitting a key-value pair, it will just grow. 

__For the purpose of this exercise we make the following assumptions:__,
- Each HBase block is only 40 bytes long (instead of the usual 64kB).
- The key of each key-value pair consists only of the row key, column qualifier and the version timestamp, as defined in the [previous exercise](#simple_keys).
- Each character takes exactly 1 byte of memory
  - For example, the first key-value pair in the diagram below would take $3 + 1 + 1 + 6 = 11$ bytes (3 bytes for `113`, 1 byte for `C`, 1 byte for `5`, 6 bytes for `little`.)

Using this information, your task is to now build the index of the HFile below:

<img src="https://polybox.ethz.ch/index.php/s/Sj5PNFWcy8TbVOh/download" width="50%">

First, calculate how the blocks will be "assembled", and then use the appropriate keys for the index.

You can use the following table (again, you can edit it by double-clicking). Use as many or as few rows as you need.

| RowId | Column | Version |
|-------|--------|---------|
|       |        |         |
|       |        |         |

What is the size of each block?

<a id='exercise_3'></a>
## Exercise 3 &mdash; Bloom filters

In [Exercise 2](#exercise_2) we built an index for an HFile containing data, and stored it in a separate index file. This index file was sorted on the row key. Each entry then mapped to the corresponding block of the HFile, whose first entry was the key-value pair with that key. 

Now, if we wanted to search for an arbitrary row key in this index, the best we could do is a binary search on the index, followed by reading 1 block from the HFile the index is for. But in the presence of many HFiles, we don't want to read all corresponding blocks, as that would greatly slow us down.

**So, how can we optimize?**

One way to do that is with **Bloom filters** - a space-efficient probabilistic data structure used by HBase to directly discard all read requests where query data is NOT stored in the HFile. Bloom Filters are stored in the metadata of each HFile.

While we did not have time to cover these filters in the lecture, they are actually crucial if we want to avoid disk reads. Bloom filters allow us to determine whether an element belongs to a set or not very quickly.

The main component of these filters is __a bit array__ with all values initially set to $0$. Let $k$ be the length of this array.

When a new element is inserted in the collection (in our case, the HFile), the following happens:
 - Its value is first run through a fixed number of hash functions, which output some numbers.
 - The locations in the bit array corresponding to the outputs from the hash functions modulo $k$ are set to 1.

When we query for a certain value:
 - We hash its value using the same hash functions, and we get several numbers as output.
 - If the value has been previously inserted in the collection, then all the locations corresponding to the hash function outputs modulo $k$ will certainly have been set to 1.
 - On the contrary, if the element hasn't been inserted previously, then the locations might or might not have already been set to 1 by other elements in the collection.
  
Thus, if prior to accessing the collection, we run our value through the hash functions, check the locations corresponding to the outputs modulo $k$, and find any of them to be 0, __we are guaranteed that the element is not present in the collection__ (No False Negatives), and we don't have to waste additional time. If the corresponding locations are all set to 1, the element may or may not be present in the collection (possibility of False Positives) and we still have to check.

For clarity, let us inspect the following sequence. Assume that we have 3 hash functions and $k = 12$. Remember that the bit array has all values initialized to $0$:
1. We insert `John Smith` in the collection. The hash functions return the locations $1$, $2$ and $7$. We set those locations in the bit array to 1
   
<img src="https://polybox.ethz.ch/index.php/s/iMEEGRSaVkRHjDG/download" width="50%">

2. We insert `Mary Smith` in the collection. The hash functions return the locations $5$, $2$ and $6$. We set those locations in the bit array to 1.
   
<img src="https://polybox.ethz.ch/index.php/s/GJQgLU8W258YsJl/download" width="50%">

3. Now, we query the collection with the key `Albert Einstein`. The hash functions return the locations $3$, $5$ and $6$. While, the bit array is set to $1$ in locations $5$ and $6$, the value at location $3$ is still $0$. This means that `Albert Einstein` __cannot__ be present in the collection. Thus we do not have to check the it.

<img src="https://polybox.ethz.ch/index.php/s/MAQM88My5yWdTps/download" width="50%">

4. We query the collection with the key `Louis de Broglie`. The hash functions return the locations $2$, $6$ and $7$. All of the corresponding bits in the array are set  to $1$, even though the key is not present in the collection. This means the filter returned a false positive and we have to traverse the collection only to find out the key is not present in there.

<img src="https://polybox.ethz.ch/index.php/s/xWrmQIPWV5iGj64/download" width="50%">

As you have seen in [Exercise 1](#exercise_1), without bloom filters, HBase has to check all HFiles, along with the MemStore, when looking for a particular key. With Bloom filters, we can avoid checking entire HFiles.

And now to the key point of this talk - how bloom filters are used in HBase. In short:

__We associate each HFile with a bloom filter. Before looking inside a particular HFile, HBase first checks the requested key against the Bloom filter associated with that HFile. If it says that the key does not exist, the file is not read. If it says the key might be inside, we proceed as usual - binary search on the index, followed by retrieving and checking the corresponding HBlock.__

You will now populate a bloom filter using a list of words. Then you will query it.

A bloom filter requires several hash functions. To keep things easily computiable, for the purpose of this exercise, we define the following 3 hash functions:

1. Given a word $x$, the value of the first hash function, $hash_1(x)$, is equal to the *first letter of the word*.
2. Given a word $x$, the value of the second hash function, $hash_2(x)$, is equal to the *second letter of the word*.
3. Given a word $x$, the value of the third hash function, $hash_3(x)$, is equal to the *third letter of the word*. 

For instance, for the word $w = $ `federal`:
- $hash_1(w) =$ `f`
- $hash_2(w) =$ `e`
- $hash_3(w) =$ `d`

As you have noticed, we use letters instead of numbers as outputs of these hash functions. Letters are also used as the locations in the bit array. This is to keep things simple.

As previously mentioned, a bloom filter starts with a bit array which has value $0$ recorded for each possible output value. To avoid clutter, we omit the value of the bit array if it is $0$. Thus, the start state of the filter is as follows:
| | | | | | | | | | | | | | | | | | | | | | | | | | |
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|

Now, if we add the word "`federal`" to the Bloom filter, using the three hash functions that we have defined above, we get the following bit array:

| | | |1|1|1| | | | | | | | | | | | | | | | | | | | |
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|

**Your task is as follows:**<br>

**1.** Populate the following table with the outputs of the three hash functions (double-click the table to edit it):
[**Collection A**]

| Word    | $hash_1$    | $hash_2$    | $hash_3$    |
|:--------|-------------|-------------|-------------|
|round    |             |             |             |
|sword    |             |             |             |
|past     |             |             |             |
|pale     |             |             |             |
|nothing  |             |             |             |
|darkness |             |             |             |
|water    |             |             |             |
|feet     |             |             |             |
|thin     |             |             |             |
|passage  |             |             |             |
|corner   |             |             |             |

__2.__ Now, *add* each word from the list to the following Bloom filter (you can also double-click to edit it) [**Bloom Filter for Collection A**]

| | | | | | | | | | | | | | | | | | | | | | | | | | |
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|

__3.__ For each word in the following list, state whether this Bloom filter reports it as belonging to the set or not. Where applicable, indicate whether the result is a false positive or not: [**Collection B**]

| Word    | $hash_1$ | $hash_2$ | $hash_3$ | The Bloom filter says the word belongs to the set: (yes/no) | False positive (yes/no/NA) |
|:--------|----------|----------|----------|:-----------------------------------------------------------:|:---------------------------|
|sword    |          |          |          |                                                             |                            |
|sound    |          |          |          |                                                             |                            |
|psychic  |          |          |          |                                                             |                            |
|pale     |          |          |          |                                                             |                            |
|book     |          |          |          |                                                             |                            |
|deaf     |          |          |          |                                                             |                            |
|truss    |          |          |          |                                                             |                            |

Which of the words that were flagged by the Bloom filter as **not** belonging to the set actually **do belong** to the set (a *false negative* outcome)?

<a id='exercise_4'></a>
## Exercise 4 &mdash; Log-structured merge-trees (LSM trees)

So far, the optimisations we have done can be summarised as follows:

1. **Sparse Index** (HFile index)  
   A small, in-memory index that stores pointers. This helps quickly locate and jumpstart the approximate position of a key in the HFile before doing a more precise binary search.

2. **Bloom Filter**  
   A **probabilistic** data structure that can definitively say if a key **does not** exist in a given HFile (no false negatives). If the Bloom filter says "not present," the HFile can be skipped entirely. If it says "might be present," a normal lookup is done.


In exercises 1 to 3 we have essentially seen methods to optimise reads on our HFiles. **But how are the HFiles created in the first place?**

Essentially, the whole HBase system composed of MemStores, HFiles, and HFile indexes is an implementation of a very common pattern called [**log-structured merge-trees**](https://en.wikipedia.org/wiki/Log-structured_merge-tree). 

A **Log-Structured Merge (LSM) Tree** is a type of database index optimized for write-intensive workloads. It relies on maintaining an in-memory data structure (often a balanced binary search tree or skip list) for fast writes and periodically flushing that data to disk in sorted files.

Before we explain how LSM Trees function, we give a mapping from common terminology to HBase specific naming:

| Common name              | HBase name |
|:-------------------------|:-----------|
| MemTable                 | MemStore   |
| SSTable                  | HFile      |
| Tombstone                | Key type   |
| Write-Ahead Log (WAL)    | HLog       |

#### 1. **In-Memory Component (MemTable)**

At the heart of an LSM Tree is an **in-memory balanced search tree** (e.g., a Red-Black Tree, AVL Tree, or sometimes a skip list). This structure, commonly called a **MemTable**, provides:

- **Fast writes:** $\mathcal{O}(\log{n})$ insertion/update in memory.
- **Fast reads on recent data:** $\mathcal{O}(\log{n})$ lookups for keys currently in memory.

However, this approach has two main issues:

1. **Lack of Durability**  
   If the system crashes, everything in memory is lost.

2. **Limited Memory and Excessive Growth**  
   The in-memory tree can grow very large and must eventually be offloaded to disk to free up RAM.

Due to the above reasons, we have to periodically flush the data to disk.

#### 2. **Flushing to SSTables**

When the in-memory tree **exceeds a certain size** threshold, the database **flushes** it to disk, creating an **SSTable (Sorted String Table)**. This is done as follows:

1. Perform an **in-order traversal** of the balanced tree to produce a sorted list in $\mathcal{O}(n)$ time.  
2. Write this sorted list to a **new SSTable file** on disk.  
3. **Clear** the in-memory tree (MemTable) to accept new writes.

**SSTables are immutable** after creation. Future changes to the same keys appear in newer SSTables.

Over time, many SSTables may accumulate on disk. **Reads** must check:

1. **In-Memory Tree**: $\mathcal{O}(\log{n})$ lookup. In case of a miss, we then proceed to check On-Disk.
2. **On-Disk SSTables**: Search from **newest** to **oldest**. Each SSTable is sorted, so it can be searched with **binary search** (O(log n) per SSTable).

If a key is found in a recent SSTable, there is no need to check older SSTables. Because SSTables are immutable, a newer table always has the most up-to-date version of a key.




#### 3. **Deletions via Tombstones**

As we have mentioned, our SSTables are immutable once written to disk. To handle deletions, LSM Trees do not immediately remove old records. Instead, a **tombstone** (a special marker) is written for a key that is deleted. The tombstone indicates that any earlier entries for that key should be ignored.

- **Old SSTables** still contain the key.  
- **The newer SSTable** has the tombstone, effectively overriding the older version.

A tombstone is similar to the idea of a **“soft delete”** from the relational database world. When we delete data, HBase does not delete it right away, but associates a tombstone with it, instead. In other words, a tombstone is a marker that is kept to indicate data that has been deleted. When we execute a delete operation it’s instead treated as an insert operation that places a tombstone on the value. Tombstones are removed as part of major compaction: during a major compaction, any row with an expired tombstone will not be propagated further.

Tombstones are a solution for deletes on LSM Trees, but they cause the following problems:

1. As a tombstone itself is a record, it takes storage space. Hence, it should be kept in mind that **upon deletion, the application will end up increasing the data size** instead of shrinking it. Furthermore, if there are a lot of tombstones, the available storage for the application could be substantially reduced.
2. When a table accumulates many tombstones, read queries on that table could become slow and can cause serious performance problems like timeouts. This is because we have to read much more data until an actual major compaction happens and removes the tombstones (and major compactions happen very infrequently, in the order of only once a week).


#### 4. **Compaction**

Since SSTables are immutable, over time:

- **Tombstoned keys** accumulate.  
- **Multiple versions** of the same key may exist in different SSTables.

**Compaction** merges these SSTables, with the following procedure:

1. **Merge Sort** approach: Because each SSTable is sorted, merging them is **$\mathcal{O}(n)$** overall (where $n$ is the combined size of the SSTables).  
2. **Discard old versions and tombstones** during the merge, creating a new, consolidated SSTable.  
3. **Remove the original SSTables**.

Compaction **reduces storage overhead** and **improves read performance** by minimizing the number of files to check.

In HBase, there are two types of compaction. 
- **Minor Compactions** usually merge a small number of HFiles in a Region, and they do not remove tombstones.
- **Major Compactions** merge **all** HFiles in a Region, and therefore can safely remove tombstones. As the operation is extremely expensive, it is usually ran infrequently (by default every seven days).

#### 5. **Your Task**

In summary, an LSM tree is highly efficient in applications using wide column storage such as HBase, Cassandra, BigTable, LevelDB, where insertions in memory happen quite often.

The following figure is from the HBase Guide book where we see how a multipage block is merged from the in-memory tree into the next on-disk tree. Trees in the store files are arranged similar to B-trees. Merging writes out a new block with the combined result. Eventually, the trees are merged into the larger blocks.<br>

<img src="https://polybox.ethz.ch/index.php/s/4o2fzmVJVbV7U1j/download" width="60%">


Inserting data into the LSM Tree:

1. When a write comes, it is inserted in the memory-resident MemStore.
2. When the size of the MemStore exceeds a certain threshold, it’s flushed to disk.
3. As MemStore is already sorted, creating a new HFile from it is efficient ($\mathcal{O}(n)$, with $n$ the total number of key-value pairs in the tree).
4. Old HFiles are periodically compacted together to save disk space and reduce fragmentation of data.

Reading data from the LSM Tree:

1. A given key is first looked up in the MemStore.
2. Using a hash index, it’s searched for in one or more HFiles, depending on the status of the compaction.

We will now walk through a concrete exercise to understand how LSM trees work in HBase. Imagine we have a client who is writing to HBase. The client also occasionally reads and deletes key-value pairs. The two types of storage that we consider in this task are the MemStore and the disk. For the purpose of this exercise, we ignore most parts of the key, apart from the row key and the timestamp. 

<img align="center" width="50%" src="https://polybox.ethz.ch/index.php/s/l1JTs41GhLE1RlK/download"><br>

The client requests the following operations (in this order):
1. write the following key-value pairs: `(C,1), (B,2), (A,9), (A,109), (G,8), (D,67), (Z,0)`;
2. read the value of key `A`;
3. delete the key `Z`;
4. write the following key-value pairs: `(S,100), (Z1,900), (A1,9), (A01,1), (Z1,850)`. 

Behind the scenes, flush and compaction operations are conducted to optimize read and write transfer. 

Please draw how the key-value pairs are stored in HBase and how HBase flushes and compacts files. Please note the following assumptions and simplifications:
   - To simplify notation, we use `key[t] value` to denote that value `value` is associated with key `key` at time `t`. For example, when the client writes the key-value pair `(C,1)`, it is represented as `C[t1] 1`. Take a look at the image below for reference:

<img align="center" width="50%" src="https://polybox.ethz.ch/index.php/s/tHMLCXeJZ2IW8Yp/download"><br>

   - The MemStore gets full after three key-value pairs have been inserted.
   - When there exist two HFiles of size $n$ on the hard drive, we compact them.
   - The timestamp is incremented by one __after__ every write. We start with a timestamp of `t1`.
   - Only the latest version of a key is kept in the MemStore.
   - When compacting, only the latest value of a key is preserved and tombstone key-value pairs are deleted.

<a id='exercise_5'></a>
## Exercise 5 &mdash; Write-Ahead Log (WAL)

In this course, we have mentioned that machines can fail or restart at any time. What happens when the machine a program is running on loses power? The program might have atomicity and durability needs. Thus, it needs to know what the last thing it was doing was. Based on this information, it might then decide to redo or undo some of the operations it previously did to restore its state to a consistent one.

So, how does the program retrieve information about what it was doing before the system crash? The answer is __logging__: persisting information about the operation on disk in append-only fashion either before or after performing it. In this exercise, we consider writing __before__ performing the operation. This type of logging is known as write-ahead logging and the corresponding log is called a **Write-ahead Log (WAL)** or **transaction log** or **commit log** or, in the case of HBase, an **HLog**. Writing to the WAL guarantees that if the machine crashes, the system will be able to recover and reapply the operation if necessary.  We have already seen one kind of WAL in this course - HDFS' edit log (or journal)!

The key idea behind the WAL is that all modifications are persisted to a log file on disk, before they are actually applied in-memory. Each log entry contains enough information to redo or undo the modification. The log can be read on every restart to recover the previous state by replaying the necessary log entries. Using WAL results in a significantly reduced number of disk writes, because only the log file needs to be flushed to disk to guarantee that a transaction is committed, rather than every data file changed by the transaction.

Each node, in a distributed environment, maintains its own log. A WAL is only appended to.

<img align="center" width="50%" src="https://polybox.ethz.ch/index.php/s/un9OrcnS0Dw7GH2/download"><br>

To ensure durability, whenever a node in HBase receives a write request, it immediately writes the data to its WAL stored on HDFS before writing the data to a MemStore. This provides durability in the case of an unexpected shutdown, since all the data on the MemStore resides on volatile memory. On startup, the WAL will be replayed to repopulate the MemStore.

<img align="center" width="50%" src="https://polybox.ethz.ch/index.php/s/5Tvn3SMUs11bhqG/download"><br>

In summary, to ensure durability, LSM Trees use a **Write-Ahead Log**:

1. **Every write** (insert/update/delete) is **appended sequentially** to the WAL on disk.
2. The same write is then applied to the in-memory tree.

If the system crashes, the WAL can be **replayed** to reconstruct the in-memory tree. Because the WAL is **sequentially** written, it minimizes random I/O overhead.

- **Pros:** Ensures durability with relatively fast sequential writes.  
- **Cons:** Requires extra disk I/O for the log (although still quite efficient).

Now, take a look at the configuration below. Note the following assumptions:
- For simplicity, keys in this exercise consist only of a row key identifier and a timestamp (indicated as `TS`)
- We use tombstone values for deleted entries (indicated as `tomb`).
- The HLog file, if present, mirrors the contents of the MemStore at the given configuration.
- When performing a read, only the latest value is returned.

<img src="https://polybox.ethz.ch/index.php/s/28M8rKQ2DrqzPnG/download"></img>

Unfortunately, at time $t = 18$, the system crashes. After a short while, it recovers.

__For each of the following keys indicate what value will be returned upon a read request of that key after the system's recovery, depending on whether the system kept a WAL before the crash or not.__

Fill in the table below (You can double-click to edit it)\
_Hint: Use `empty` to indicate a missing return value_


| Key      |  Value with HLog  | Value without HLog |
|:--------:|:-----------------:|:------------------:|
| bat      |                   |                    |
| bear     |                   |                    |
| fox      |                   |                    |
| monkey   |                   |                    |
| zebra    |                   |                    |